# Naive Bayes Classifier

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the dataset

In [ ]:
df = pd.read_csv("emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [ ]:
df.columns
df.shape

(5728, 2)

In [ ]:
#Checking for duplicates and removing them
df.drop_duplicates(inplace = True)
df.shape

(5695, 2)

In [ ]:
#Checking for any null entries in the dataset
print (pd.DataFrame(df.isnull().sum()))

      0
text  0
spam  0


In [ ]:
# Using NLP to clean the dataset and convert to one corpus
# Cleaning the texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
#Every mail starts with 'Subject :' will remove this from each text
df['text']=df['text'].map(lambda text: text[1:])
df['text'] = df['text'].map(lambda text:re.sub('[^a-zA-Z0-9]+', ' ',text)).apply(lambda x: (x.lower()).split())
ps = PorterStemmer()
nltk.download('stopwords')
corpus=df['text'].apply(lambda text_list:' '.join(list(map(lambda word:ps.stem(word),(list(filter(lambda text:text not in set(stopwords.words('english')),text_list)))))))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df.head()

,text,spam
0,"[ubject, naturally, irresistible, your, corpor...",1
1,"[ubject, the, stock, trading, gunslinger, fann...",1
2,"[ubject, unbelievable, new, homes, made, easy,...",1
3,"[ubject, 4, color, printing, special, request,...",1
4,"[ubject, do, not, have, money, get, software, ...",1


In [ ]:
corpus

0       ubject natur irresist corpor ident lt realli h...
1       ubject stock trade gunsling fanni merril muzo ...
2       ubject unbeliev new home made easi im want sho...
3       ubject 4 color print special request addit inf...
4       ubject money get softwar cd softwar compat gre...
                              ...                        
5723    ubject research develop charg gpg forward shir...
5724    ubject receipt visit jim thank invit visit lsu...
5725    ubject enron case studi updat wow day super th...
5726    ubject interest david pleas call shirley crens...
5727    ubject news aurora 5 2 updat aurora version 5 ...
Name: text, Length: 5695, dtype: object

In [ ]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus.values).toarray()
y = df.iloc[:, 1].values

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
# Fitting Naive Bayes classifier to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
classifier.fit(X_train , y_train)

MultinomialNB()

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

array([[867,  11],
       [  1, 260]])

In [ ]:
#this function computes subset accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9894644424934153

In [ ]:
accuracy_score(y_test, y_pred,normalize=False)

1127

In [ ]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
accuracies.mean()
accuracies.std()

0.004192656232767988